# Python을 이용해 CNN 구현하기

#### 2018-02-01 오상신

---

## 들어가기 전에
---

#### 이 노트북에서 구현하는 코드는 [출처](https://tensorflow.blog/5-텐서플로우-다중-레이어-뉴럴-네트워크-first-contact-with-tensorflow/)를 참고했습니다.

#### 코드
- 이 노트북에서 구현하고자 하는 것은 **Convolutional Neural Network**를 이용해 MNIST 데이터를 분류하는 코드입니다.
- **Tensorflow**를 이용해 구현하며, 따라서 backpropagation은 직접 구현하지 않습니다.

#### Jupyter Notebook
- 이 노트북에서 직접 코드를 실행시킬 수 있으며, 단축키는 **shift + enter**입니다.
- **A**키나 **B**키를 이용해서 현재 cell의 위 혹은 아래에 새로운 cell을 추가할 수 있습니다.
- 이 노트북에서 직접 documentation을 확인할 수 있으며, 그 단축키는 **shift + tab**입니다.

#### * 기타 단축키들:
- **A** or **B** : 현재 셀 위 혹은 아래에 새로운 셀을 추가합니다.
- **DD** : 현재 셀을 삭제합니다.
- **Z** : 삭제한 셀을 다시 생성합니다.
- **Y** : 현재 셀을 코드를 실행시킬 수 있는 셀로 변환합니다.
- **M** : 현재 셀을 Markdown 셀로 변환합니다.

---

## 순서
---
이 튜토리얼에서 **CNN**을 이용해 *MNIST* dataset을 분류하는 작업을 하게 됩니다.

이 작업은 간단하게 분류하면 다음과 같은 순서로 진행됩니다.

- MNIST 데이터베이스 다운로드 및 읽기
- CNN 모델 제작
- 모델 학습
- 학습 결과 확인 및 출력

기타 필요한 설명들은 코드 작성과 함께 진행하겠습니다.

---

## MNIST dataset 받기
---
Tensorflow는 MNIST를 인터넷에서 받아올 수 있는 간단한 코드를 제공합니다.

이를 이용해서 dataset을 원하는 디렉토리에 다운로드 받아 봅시다.

In [45]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### MNIST dataset이란?
---
[Yann LeCun 개인 홈페이지](http://yann.lecun.com/exdb/mnist/)에서 더 많은 자료를 볼 수 있습니다.

MNIST dataset은 필기체 숫자를 모아놓은 이미지 데이터셋으로 여러 튜토리얼에서 딥러닝을 시작할 때 소개하는 필기체 숫자 분류를 위한 데이터셋입니다.
6만 개의 training 데이터와 1만 개의 test 데이터로 구분되어 있으며, 다음과 같습니다.

* train-images-idx3-ubyte.gz : training용 이미지 데이터
* train-labels-idx1-ubyte.gz : training용 라벨 데이터
* t10k-images-idx3-ubyte.gz  : test용 이미지 데이터
* t10k-labels-idx1-ubyte.gz  : test용 라벨 데이터

training용 데이터와 test용 데이터 사이에 구조적인 차이점은 없으며, 이미지 데이터는 *784(=28X28)*픽셀의 그레이 스케일 데이터이고, 라벨 데이터는 0부터 9까지의 숫자로 되어있습니다. 자세한 포맷은 위 링크 하단에 설명되어 있습니다.

---

## CNN 모델 구성

#### Tensorflow import 및 세션 열기
---
Tensorflow를 사용하기 위해서는 우선 패키지를 *import*해야 합니다.

기본적으로 tensorflow는 모델을 먼저 구성한 이후에 실질적인 연산 과정을 시작합니다.

모델을 구성한 이후에 Tensorflow에서 제공하는 연산을 위해서는 *Session*이라고 하는 클래스 변수가 필요합니다.([tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session))

In [46]:
import tensorflow as tf

sess = tf.InteractiveSession()

#### Input과 Target data를 위한 placeholder
---
모델을 구성할 때, 입력과 타겟에 쓸 데이터를 위한 일종의 변수를 지정해 두어야 하는데, 이를 *placeholder*라 합니다.

Output 데이터는 입력과 모델을 통해 계산할 값이므로 필요하지 않습니다.

이 때, 입력은 앞서 말한대로 28&#42;28&#42;1 = 784의 차원을 가지며, 한 번에 여러 이미지를 동시에 처리할 예정이기 때문에 **(None, 784)**차원의 *placeholder*를 생성합니다.

마찬가지로 출력은 0부터 9까지의 값만을 갖고, [one-hot vector](https://en.wikipedia.org/wiki/One-hot)의 형태를 가지기 때문에 **(None, 10)**차원의 *placeholder*를 생성합니다.

In [47]:
X = tf.placeholder(tf.float32, shape=[None, 784])
    # float32는 타입, shape은 차원
y = tf.placeholder(tf.float32, shape=[None, 10])

#### Input data reshape하기
입력 데이터가 1차원으로 쭉 늘어져 있는 형태이기 때문에 *(높이)&#42;(너비)&#42;(색 차원)*의 형태로 reshape 해줍니다.

이 때, 한 번에 여러 데이터를 처리할 것이므로 맨 앞에 데이터 수에 대한 정보를 입력해주면 됩니다.

In [48]:
# -1은 자동으로 shape 모양을 정해주는 것
X_image = tf.reshape(X, [-1, 28, 28, 1])

---

#### kernel과 bias 변수 생성하기
---
CNN에서는 kernel이 weight의 역할을 합니다.

kernel의 shape을 변수로 받아 kernel와 bias를 동시에 반환하는 함수를 작성해봅시다.

In [49]:
def initial_value(shape) :
    
    '''
    shape을 받아서 차원에 맞는 Kernel 변수와 Bias 변수를 반환하는 함수
    
    1. Shape을 리스트로 받아서 shape_kernel에 그대로 할당
    2. Bias의 차원과 shape의 마지막 값이 같음 (constant는 리스트를 받아서 리스트로 감쌌다)
    3. Truncated_normal (양쪽이 잘린 정규분포)로 initialize
    
    '''
    shape_kernel = shape
    shape_bias = [shape[-1]]
    
    initial_kernel = tf.truncated_normal(shape = shape_kernel, stddev = 0.1)
    initial_bias = tf.constant(0.1, shape = shape_bias)
    
    return tf.Variable(initial_kernel), tf.Variable(initial_bias)

이제, 만들어진 kernel과 bias를 이용해 convolution, activation function(ReLU), pooling을 수행하는 함수를 작성해봅시다.

In [50]:
def get_next_layer(x, kernel, bias) :
    
    '''
    다음 layer를 찾는 함수
    
    '''
    
    # 1. Convolution
    conv = tf.nn.conv2d(x, kernel, strides=[1, 1, 1, 1], padding= 'SAME')
        # x와 kernel을 convolution. stride는 step을 의미. padding은 주변을 0으로 채워서 사이즈 유지하는 것.
    
    
    # 2. ReLu
    relu = tf.nn.relu(conv + bias)
    
    
    # 3. Max Pooling
    pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding= 'SAME')
        # 2x2에서 가장 큰 값을 Pool, ksize에서 마지막 1은 색 차원을 의미함
        # ksize를 맞지 않을 경우, strides와 padding을 통해 해결함
        
    return pool

---

### 모델에 대한 이해
---
전체적인 모습은 convolution layer 2개, fully-connected layer 2개를 통과시키는 모델입니다.

그 중 첫번째 convolution layer를 먼저 설명드리겠습니다.

---
#### convolution layer

우선 입력되는 데이터는 앞서 말한 바와 같이, (*28&#42;28&#42;1*)의 형태입니다. 

여기에 (*5&#42;5&#42;**1**&#42;32*)의 kernel과 **zero-padding**을 이용해 **convolution**을 수행하고, 여기에 bias를 더한 뒤, activation function(**ReLU**)을 통과시킵니다. 

그 후, (*2&#42;2*) **max-pooling**을 수행하면 결과적으로 (*14&#42;14&#42;32*) 차원의 tensor가 생성됩니다.

그림으로 표현하면 다음과 같습니다.

![First_layer](./IMG_0169.jpg)

첫번째 convolutional layer를 통과한 이후, 같은 작업을 (***5&#42;5&#42;32&#42;64***) 의 차원을 갖는 kernel을 이용해 수행하게 되면 (*7&#42;7&#42;64*) 차원의 tensor가 만들어지게 됩니다.

이제 그 연산을 위한 변수들 먼저 설정해봅시다.

In [51]:
depth = [1, 32, 64]
k_conv, b_conv = [], []
h_conv = [X_image]

위의 변수들과 앞서 작성한 함수들을 활용해 convolution layer 두 개를 작성해봅시다.

In [52]:
# 첫번째 1, 32의 depth
# 두번째 32, 64의 depth

for i in range(2) :
    tmp_v, tmp_b = initial_value([5, 5, depth[i], depth[i+1]])
    k_conv.append(tmp_v)
    b_conv.append(tmp_b)
    
    h_conv.append(get_next_layer(h_conv[i], k_conv[i], b_conv[i]))
    
    
# 결과는 7 x 7 x 64 의 텐서가 만들어짐

---

#### fully connected layer
---
이제 fully connected layer를 작성할 차례입니다.

이 작업을 위해서 (*7&#42;7&#42;64*) 차원의 tensor를 길게 펼친 뒤, fully connected layer를 이용해 1024개짜리 vector로 만들어줍니다.

그 후, 두 번째 layer를 이용해 10-vector를 만들어 타겟과 비교할 것입니다.

우선 convolution layer의 마지막 층을 vector로 변환하고, 작업들을 위한 변수를 설정해봅시다.

In [53]:
h_fc = [tf.reshape(h_conv[-1], [-1, 7*7*64])]

dim_fc = [7*7*64, 1024, 10]
weights, biases = [], []

위의 변수들을 이용해서 fully connected layer를 계산해봅시다.

In [54]:
for i in range(2) :
    tmp_v, tmp_b = initial_value([dim_fc[i], dim_fc[i+1]])
    weights.append(tmp_v)
    biases.append(tmp_b)

h_fc.append(tf.nn.relu(tf.matmul(h_fc[-1], weights[0]) + biases[0]))

1024라는 숫자는 아무래도 큰 것 같으니 dropout을 이용해서 overfitting 문제를 예방해봅시다.

dropout 확률은 학습시와 정확도 계산시 변경할 수 있도록 placeholder를 이용해 지정합시다.

In [55]:
# 1024에서 10으로 가면 그 변화가 너무 크다
# tf에서 제공하는 dropout function을 사용

keep_prob = tf.placeholder(tf.float32)
    # keep 할 확률을 정함
h_fc.append(tf.nn.dropout(h_fc[-1], keep_prob))

이제 softmax함수를 이용해서 타겟과 비교할 y를 만들어봅시다.

In [56]:
y_conv = tf.nn.softmax(tf.matmul(h_fc[-1], weights[1]) +biases[1])

---

#### loss function 및 정확도 계산하기
---
이제 학습을 위해 cross entropy를 구하고, optimizer를 만들어봅시다.

In [57]:
cross_entrophy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_conv), reduction_indices=[1]))
    #reduction_indices는 더할 축을 결정함
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entrophy)

그리고 그 결과를 출력하기 위해 정확도를 계산하는 코드를 작성해봅시다.

우선 타겟과 출력 data에서 가장 큰 값이 어디있는지 *argmax*함수를 통해 찾은 뒤, 그 두 값이 같은지 다른지 boolean값으로 판별합니다.

그 후, 그 값들을 1과 0으로 type casting하여 평균을 구하면, 우리가 원하는 정확도 값을 구할 수 있습니다.

In [58]:
correct_predictions = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    # T/F 반환
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    # float으로 casting함 (True -> 1, False -> 0)

---

## 학습 진행하기
---
이제 모델은 완성되었습니다!

이제 남은 일은 변수를 초기화하고 학습을 진행시키는 것입니다.

#### 변수 초기화하기
---
변수를 초기화하는 것은 세션을 이용해 간단히 끝낼 수 있습니다.

In [62]:
sess.run(tf.global_variables_initializer())
    # 모든 변수 초기화

#### 학습 진행하기
---
반복문을 이용해 학습을 진행해봅시다.

mini batch의 크기는 50이고, 따라서 1200번마다 모든 데이터를 학습에 이용하게 됩니다.

In [65]:
for i in range(601) :
    batch = mnist.train.next_batch(50)   #50개씩 받아온다. 메모리에 따라 키워도 된다.
    
    if i != 0 :
        train_step.run(feed_dict={X: batch[0], y : batch[1], keep_prob:0.5})
        
    if i%120 == 0 :
        train_accuracy = accuracy.eval(feed_dict={ X: batch[0], y : batch[1], keep_prob : 1.0} )
        print("step %d, training accuracy %g" % (i, train_accuracy))

step 0, training accuracy 0.06
step 120, training accuracy 0.84
step 240, training accuracy 0.92
step 360, training accuracy 0.94
step 480, training accuracy 0.94
step 600, training accuracy 0.94


---

## 결과 확인하기
---
이제 학습이 모두 끝났고, 남은 것은 test dataset을 이용하여 얼마나 좋은 정확도를 확인하는 것입니다.

In [66]:
print("test accuracy %g" % (accuracy.eval(feed_dict={X:mnist.test.images, y : mnist.test.labels, keep_prob : 1.0})))

test accuracy 0.9469
